# Self-Driving Car Engineer Nanodegree

## Deep Learning

## Project: Build a Traffic Sign Recognition Classifier

In this notebook, a template is provided for you to implement your functionality in stages, which is required to successfully complete this project. If additional code is required that cannot be included in the notebook, be sure that the Python code is successfully imported and included in your submission if necessary. 

> **Note**: Once you have completed all of the code implementations, you need to finalize your work by exporting the iPython Notebook as an HTML document. Before exporting the notebook to html, all of the code cells need to have been run so that reviewers can see the final implementation and output. You can then export the notebook by using the menu above and navigating to  \n",
    "**File -> Download as -> HTML (.html)**. Include the finished document along with this notebook as your submission. 

In addition to implementing code, there is a writeup to complete. The writeup should be completed in a separate file, which can be either a markdown file or a pdf document. There is a [write up template](https://github.com/udacity/CarND-Traffic-Sign-Classifier-Project/blob/master/writeup_template.md) that can be used to guide the writing process. Completing the code template and writeup template will cover all of the [rubric points](https://review.udacity.com/#!/rubrics/481/view) for this project.

The [rubric](https://review.udacity.com/#!/rubrics/481/view) contains "Stand Out Suggestions" for enhancing the project beyond the minimum requirements. The stand out suggestions are optional. If you decide to pursue the "stand out suggestions", you can include the code in this Ipython notebook and also discuss the results in the writeup file.


>**Note:** Code and Markdown cells can be executed using the **Shift + Enter** keyboard shortcut. In addition, Markdown cells can be edited by typically double-clicking the cell to enter edit mode.

---
## Step 0: Load The Data

In [1]:
# import required packages
import pickle
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import random
from mpl_toolkits import mplot3d
from tensorflow.contrib.layers import flatten
import tensorflow as tf
from sklearn.utils import shuffle


%matplotlib inline


/Users/fchew/anaconda3/envs/carnd-term1/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:458: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/Users/fchew/anaconda3/envs/carnd-term1/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:459: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/Users/fchew/anaconda3/envs/carnd-term1/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:460: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/Users/fchew/anaconda3/envs/car

In [2]:
# Load pickled data
training_file  = "data/train.p"
validation_file= "data/valid.p"
testing_file   = "data/test.p"

with open(training_file, mode='rb') as f:
    train = pickle.load(f)
with open(validation_file, mode='rb') as f:
    valid = pickle.load(f)
with open(testing_file, mode='rb') as f:
    test = pickle.load(f)
    
X_train, y_train = train['features'], train['labels']
X_valid, y_valid = valid['features'], valid['labels']
X_test, y_test = test['features'], test['labels']

FileNotFoundError: [Errno 2] No such file or directory: 'data/train.p'

---

## Step 1: Dataset Summary & Exploration

The pickled data is a dictionary with 4 key/value pairs:

- `'features'` is a 4D array containing raw pixel data of the traffic sign images, (num examples, width, height, channels).
- `'labels'` is a 1D array containing the label/class id of the traffic sign. The file `signnames.csv` contains id -> name mappings for each id.
- `'sizes'` is a list containing tuples, (width, height) representing the original width and height the image.
- `'coords'` is a list containing tuples, (x1, y1, x2, y2) representing coordinates of a bounding box around the sign in the image. **THESE COORDINATES ASSUME THE ORIGINAL IMAGE. THE PICKLED DATA CONTAINS RESIZED VERSIONS (32 by 32) OF THESE IMAGES**

Complete the basic data summary below. Use python, numpy and/or pandas methods to calculate the data summary rather than hard coding the results. For example, the [pandas shape method](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.shape.html) might be useful for calculating some of the summary results. 

### Provide a Basic Summary of the Data Set Using Python, Numpy and/or Pandas

In [ ]:
# Number of training examples
n_train = len(X_train)

# Number of validation examples
n_validation = len(X_valid)

# Number of testing examples.
n_test = len(X_test)

# What's the shape of a traffic sign image?
image_shape = X_train[0].shape

# How many unique classes/labels there are in the dataset.
n_classes = len(np.unique(y_train))

print("Number of training examples =", n_train)
print("Number of testing examples =", n_test)
print("Image data shape =", image_shape)
print("Number of classes =", n_classes)

### Include an exploratory visualization of the dataset

Visualize the German Traffic Signs Dataset using the pickled file(s). This is open ended, suggestions include: plotting traffic sign images, plotting the count of each sign, etc. 

The [Matplotlib](http://matplotlib.org/) [examples](http://matplotlib.org/examples/index.html) and [gallery](http://matplotlib.org/gallery.html) pages are a great resource for doing visualizations in Python.

**NOTE:** It's recommended you start with something simple first. If you wish to do more, come back to it after you've completed the rest of the sections. It can be interesting to look at the distribution of classes in the training, validation and test set. Is the distribution the same? Are there more examples of some classes than others?

In [ ]:
signName = pd.read_csv('signnames.csv')

# plot a random sampling of 20 images
plt.figure(1, figsize=(12,8))
plt.subplots_adjust(wspace=0.7,hspace=0.1)

for i in range(12):
    idx = random.randint(0,len(X_train))
    plt.subplot(3, 4, i+1)
    plt.imshow(X_train[idx])
    plt.title(str(y_train[idx])+": "+str(signName.loc[y_train[idx]].SignName), fontsize=9)
    plt.axis('off')

In [ ]:
# print(y_train[1:10])
# hist, bins = np.histogram(y_train, bins=n_classes)
# print(hist)
# print(bins)

In [ ]:
# visualise data samples as bar charts
f, (x1, x2, x3) = plt.subplots(1,3, figsize=(32,25))

hist, bins = np.histogram(y_train, bins=n_classes)
colors = plt.get_cmap('RdYlGn')(np.linspace(0.15, 0.85, n_classes))
height = 0.8*(bins[1]-bins[0])
center = (bins[:-1]+bins[1:])/2
# f.plt.set_ylabel('Sign Name')
# x1.barh(bins,hist)
x1.barh(center,hist,height,tick_label=signName.SignName,color=colors)
x1.tick_params(axis="y", labelsize=30)
x1.tick_params(axis="x", labelsize=25)
x1.set_title('Training Data',fontsize=30)
x1.grid(True)
x1.set_xlabel('Number of Data Samples', fontsize=30)

hist, bins = np.histogram(y_valid, bins=n_classes)
# x2.barh(y=y_valid,width=n_classes)
x2.barh(center,hist,height,color=colors)
x2.set_title('Validation Data',fontsize=30)
x2.tick_params(axis="y", labelsize=0)
x2.tick_params(axis="x", labelsize=25)
x2.grid(True)
x2.set_xlabel('Number of Data Samples', fontsize=30)

hist, bins = np.histogram(y_test, bins=n_classes)
# x3.barh(y=y_test,width=n_classes)
x3.barh(center,hist,height,color=colors)
x3.set_title('Test Data',fontsize=30)
x3.tick_params(axis="y", labelsize=0)
x3.tick_params(axis="x", labelsize=25)
x3.grid(True)
x3.set_xlabel('Number of Data Samples', fontsize=30)

In [ ]:
# visualise data samples as bar charts on one 3d projection 
fig = plt.figure(figsize=(32,18))
# f, (x1, x2, x3) = plt.subplots(1,3, figsize=(32,25))
x1 = fig.add_subplot(111, projection='3d')
bins  = n_classes

# # for z in [30, 20, 10, 0]:
# for z in [30]:
#     yVals, bins = np.histogram(y_train, bins=n_classes)
#     xVals       = (bins[:-1]-bins[1:])/2
#     ax.bar(xVals,yVals,zs=z,zdir='y',width=height,tick_label=signName.SignName,color=colors[::-1])
#     ax.view_init(azim=135,elev=45)
#     ax.axes.tick_params(axis='x',rotation=25, labelsize='small')

hist, bins = np.histogram(y_train, bins=n_classes)
colors = plt.get_cmap('RdYlGn')(np.linspace(0.15, 0.85, n_classes))
height = 0.8*(bins[1]-bins[0])
center = (bins[:-1]+bins[1:])/2
x1.bar(center,hist[::-1],zs=30,tick_label=signName.SignName[::-1],color=colors[::-1],zdir="y")
x1.tick_params(axis="y", labelsize=25)
# x1.set_title('Training Data',fontsize=30)
x1.grid(True)
x1.axes.tick_params(axis='x',rotation=90, labelsize=20)

hist, bins = np.histogram(y_test, bins=n_classes)
x1.bar(center,hist[::-1],zs=10,tick_label=signName.SignName[::-1],color=colors[::-1],zdir="y")
x1.axes.tick_params(axis='x',rotation=90, labelsize=20)

hist, bins = np.histogram(y_valid, bins=n_classes)
x1.bar(center,hist[::-1],zs=20,tick_label=signName.SignName[::-1],color=colors[::-1],zdir="y")
x1.axes.tick_params(axis='x',rotation=90, labelsize=20)
x1.axes.tick_params(axis='z', labelsize=20, direction="out", rotation=0)
x1.axes.tick_params(axis='y', labelsize=20, direction="out")
x1.axes.set_yticks([10,20,30])
x1.axes.set_yticklabels(['Validation Data','Test Data','Training Data'])

# x1.axes.set_xlabel('SignNames',fontsize=30)
x1.set_xlim(0, n_classes-1)
# ax.set_ylim(0, 10)
# ax.set_zlim(0, 10)
x1.axes.set_zlabel('Number of Samples of Training Data',fontsize=20)



----

## Step 2: Design and Test a Model Architecture

Design and implement a deep learning model that learns to recognize traffic signs. Train and test your model on the [German Traffic Sign Dataset](http://benchmark.ini.rub.de/?section=gtsrb&subsection=dataset).

The LeNet-5 implementation shown in the [classroom](https://classroom.udacity.com/nanodegrees/nd013/parts/fbf77062-5703-404e-b60c-95b78b2f3f9e/modules/6df7ae49-c61c-4bb2-a23e-6527e69209ec/lessons/601ae704-1035-4287-8b11-e2c2716217ad/concepts/d4aca031-508f-4e0b-b493-e7b706120f81) at the end of the CNN lesson is a solid starting point. You'll have to change the number of classes and possibly the preprocessing, but aside from that it's plug and play! 

With the LeNet-5 solution from the lecture, you should expect a validation set accuracy of about 0.89. To meet specifications, the validation set accuracy will need to be at least 0.93. It is possible to get an even higher accuracy, but 0.93 is the minimum for a successful project submission. 

There are various aspects to consider when thinking about this problem:

- Neural network architecture (is the network over or underfitting?)
- Play around preprocessing techniques (normalization, rgb to grayscale, etc)
- Number of examples per label (some have more than others).
- Generate fake data.

Here is an example of a [published baseline model on this problem](http://yann.lecun.com/exdb/publis/pdf/sermanet-ijcnn-11.pdf). It's not required to be familiar with the approach used in the paper but, it's good practice to try to read papers like these.

### Pre-process the Data Set (normalization, grayscale, etc.)

Minimally, the image data should be normalized so that the data has mean zero and equal variance. For image data, `(pixel - 128)/ 128` is a quick way to approximately normalize the data and can be used in this project. 

Other pre-processing steps are optional. You can try different techniques to see if it improves performance. 

Use the code cell (or multiple code cells, if necessary) to implement the first step of your project.

In [ ]:
### Preprocess the data here. It is required to normalize the data. Other preprocessing steps could include 
### converting to grayscale, etc.
### Feel free to use as many code cells as needed.

In [ ]:
# pre-process the images by grayscale and normalising
def preProcess(origImg):
    #convert to grayscale
    gray  = np.sum(origImg/3,axis=3,keepdims=True)
    norms = (gray-np.float32(128))/np.float32(128)
    return norms

X_train_process = preProcess(X_train)
X_valid_process = preProcess(X_valid)
X_test_process  = preProcess(X_test)

In [ ]:
print("              "+str(X_train.dtype)+"      "+str(X_train_process.dtype))
print("X_train_min:    "+str(np.min(X_train))+str("   --->  %.2f" % np.min(X_valid_process))+" (normalised)")
print("X_train_max:   "+str(np.max(X_train))+str("  --->   %.2f" % np.max(X_valid_process))+" (normalised)")
print(str("X_train_mean: %.2f" % np.mean(X_train))+str(" --->  %.2f" % np.mean(X_train_process))+" (normalised)")
print()
print("              "+str(X_valid.dtype)+"      "+str(X_train_process.dtype))
print("X_valid_min:    "+str(np.min(X_valid))+str("   --->  %.2f" % np.min(X_valid_process))+" (normalised)")
print("X_valid_max:   "+str(np.max(X_valid))+str("  --->   %.2f" % np.max(X_valid_process))+" (normalised)")
print(str("X_valid_mean: %.2f" % np.mean(X_valid))+str(" --->  %.2f" % np.mean(X_valid_process))+" (normalised)")
print()
print("              "+str(X_test.dtype)+"      "+str(X_train_process.dtype))
print("X_test_min:     "+str(np.min(X_test))+str("   --->  %.2f" % np.min(X_test_process))+" (normalised)")
print("X_test_max:    "+str(np.max(X_test))+str("  --->   %.2f" % np.max(X_test_process))+" (normalised)")
print(str("X_test_mean:  %.2f" % np.mean(X_test))+str(" --->  %.2f" % np.mean(X_test_process))+" (normalised)")
print()
print("X_train.shape            X_train_process.shape")
print(str(X_train.shape)+"  ---> "+str(X_train_process.shape))
print()

In [ ]:
# f, (x1, x2) = plt.subplots(1,2, figsize=(3,3))

# x1.axis('off')
# x1.imshow(X_train[100])

# x2.imshow(X_train_process[100].squeeze(),cmap='gray')
# x2.axis('off')

# plt.figure(1, figsize=(48,48))
plt.subplots(3,8, figsize=(18,12))
plt.subplots_adjust(wspace=0.4,hspace=0.1)
for i in range(0,24,3):
    idx = random.randint(0,len(X_train))
    plt.subplot(4, 6, i+1)
    plt.imshow(X_train[idx])
    plt.title(str(y_train[idx])+": "+str(signName.loc[y_train[idx]].SignName), fontsize=12)
    plt.axis('off')
    plt.subplot(4, 6, i+2)
    plt.imshow(X_train_process[idx].squeeze(),cmap='gray')
    plt.title("-->(post-processed)", fontsize=12)
    plt.axis('off')
    plt.subplot(4, 6, i+3)
    plt.axis('off')    
    i+=1
    


### Model Architecture

In [ ]:
### Define your architecture here.
### Feel free to use as many code cells as needed.

Making use of the classic LeNet Architecture implemented in the Labs during the lessons, the dataset was implementted in greyscale. 

In [ ]:
# classical LeNet-5 Architecture with dropouts
def LeNet(x):    
    # Arguments used for tf.truncated_normal, randomly defines variables for the weights and biases for each layer
    mu = 0
    sigma = 0.1
    
    # Layer 1: Convolutional. Input = 32x32x1. Output = 28x28x6.
    w1    = tf.Variable(tf.truncated_normal(shape=(5, 5, 1, 6), mean = mu, stddev = sigma))
    b1    = tf.Variable(tf.zeros(6))
    conv1 = tf.add(tf.nn.conv2d(x, w1, strides=[1, 1, 1, 1], padding='VALID'), b1)

    # Activation.
    conv1 = tf.nn.relu(conv1)

    # Pooling. Input = 28x28x6. Output = 14x14x6.
    conv1 = tf.nn.max_pool(conv1, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='VALID')

    # Layer 2: Convolutional. Output = 10x10x16.
    w2    = tf.Variable(tf.truncated_normal(shape=(5, 5, 6, 16), mean = mu, stddev = sigma))
    b2    = tf.Variable(tf.zeros(16))
    conv2 = tf.add(tf.nn.conv2d(conv1, w2, strides=[1, 1, 1, 1], padding='VALID'), b2)
    
    # Activation.
    conv2 = tf.nn.relu(conv2)

    # Pooling. Input = 10x10x16. Output = 5x5x16.
    conv2 = tf.nn.max_pool(conv2, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='VALID')

    # Flatten. Input = 5x5x16. Output = 400.
    flatConv2 = flatten(conv2)
    
    # Layer 3: Fully Connected. Input = 400. Output = 120.
    w3  = tf.Variable(tf.truncated_normal(shape=(400, 120), mean = mu, stddev = sigma))
    b3  = tf.Variable(tf.zeros(120))
    fc3 = tf.add(tf.matmul(flatConv2, w3), b3)
    
    # Activation then dropOut. (Of Layer 3: Fully Connected)
#     fc3 = tf.nn.relu(fc3)
    fc3 = tf.nn.dropout(tf.nn.relu(fc3), keepProb)

    # Layer 4: Fully Connected. Input = 120. Output = 84.
    w4  = tf.Variable(tf.truncated_normal(shape=(120, 84), mean = mu, stddev = sigma))
    b4  = tf.Variable(tf.zeros(84))
    fc4 = tf.add(tf.matmul(fc3, w4), b4)
    
    # Activation then dropout. (Of Layer 4: Fully Connected)
    fc4    = tf.nn.dropout(tf.nn.relu(fc4), keepProb)
    
    # Layer 5: Fully Connected. Input = 84. Output = 43.
    w5  = tf.Variable(tf.truncated_normal(shape=(84, n_classes), mean = mu, stddev = sigma))
    b5  = tf.Variable(tf.zeros(n_classes))
    logits = tf.add(tf.matmul(fc4, w5), b5)
    
    return logits

In [ ]:
tf.reset_default_graph() 

epochs = 40
batchSize = 128
learnRate = 0.001
dropouts = 0.5



In [ ]:
# Placeholders for input data, features 
# for RGB:
# x = tf.placeholder(tf.float32,(None,32,32,3))
# for Grayscale:
x = tf.placeholder(tf.float32,(None,32,32,1))
y = tf.placeholder(tf.int32,(None))

# Function of Y that one-hot encodes data
oneHotY = tf.one_hot(y,n_classes)
                   
# Placeholder for dropout probability in modLeNet
keepProb = tf.placeholder(tf.float32)

logits = LeNet(x)
crossEntropy = tf.nn.softmax_cross_entropy_with_logits(labels=oneHotY, logits=logits)
lossOperation = tf.reduce_mean(crossEntropy)
optimizer = tf.train.AdamOptimizer(learning_rate = learnRate)
training_operation = optimizer.minimize(lossOperation)


In [ ]:
# Model Eval - from LeNet Lab
correct_prediction = tf.equal(tf.argmax(logits, 1), tf.argmax(oneHotY, 1))
accuracy_operation = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
saver = tf.train.Saver()

def evaluate(X_data, y_data):
    num_examples = len(X_data)
    total_accuracy = 0
    sess = tf.get_default_session()
    for offset in range(0, num_examples, batchSize):
        batch_x, batch_y = X_data[offset:offset+batchSize], y_data[offset:offset+batchSize]
#         accuracy = sess.run(accuracy_operation, feed_dict={x: batch_x, y: batch_y})
        accuracy = sess.run(accuracy_operation, feed_dict={x: batch_x, y: batch_y, keepProb: 1.0})
        total_accuracy += (accuracy * len(batch_x))
    return total_accuracy / num_examples

### Train, Validate and Test the Model

A validation set can be used to assess how well the model is performing. A low accuracy on the training and validation
sets imply underfitting. A high accuracy on the training set but low accuracy on the validation set implies overfitting.

In [ ]:
### Train your model here.
### Calculate and report the accuracy on the training and validation set.
### Once a final model architecture is selected, 
### the accuracy on the test set should be calculated and reported as well.
### Feel free to use as many code cells as needed.

In [ ]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    num_examples = len(X_train)
    
    print("Training...")
    print()
    for i in range(epochs):
        X_train_process, y_train = shuffle(X_train_process, y_train)
        for offset in range(0, num_examples, batchSize):
            end = offset + batchSize
            batch_x, batch_y = X_train_process[offset:end], y_train[offset:end]
            sess.run(training_operation, feed_dict={x: batch_x, y: batch_y, keepProb: dropouts})
            
        validation_accuracy = evaluate(X_valid_process, y_valid)
        print("EPOCH {} ...".format(i+1))
        print("Validation Accuracy = {:.3f}".format(validation_accuracy))
        print()
    print("=========")
    test_accuracy = evaluate(X_test_process, y_test)
    print("Test Accuracy = {:.3f}".format(test_accuracy))
    print("Validation Accuracy = {:.3f}".format(validation_accuracy))
    train_accuracy = evaluate(X_train_process, y_train)
    print("Train Accuracy = {:.3f}".format(train_accuracy))
    print("=========")
    saver.save(sess, 'lenet')
    print("Model saved")

In [ ]:
# # model performance - from LeNet Lab
# with tf.Session() as sess:
#     sess.run(tf.global_variables_initializer())
#     saver = tf.train.import_meta_graph('lenet.meta')
#     saver.restore(sess, "lenet")
    
    
#     saver.restore(sess, tf.train.latest_checkpoint('.'))

#     test_accuracy = evaluate(X_test_process, y_test)
#     print("Test Accuracy = {:.3f}".format(test_accuracy))
    
#     train_accuracy = evaluate(X_train_process, y_train)
#     print("Train Accuracy = {:.3f}".format(train_accuracy))


---

## Step 3: Test a Model on New Images

To give yourself more insight into how your model is working, download at least five pictures of German traffic signs from the web and use your model to predict the traffic sign type.

You may find `signnames.csv` useful as it contains mappings from the class id (integer) to the actual sign name.

### Load and Output the Images

In [ ]:
### Load the images and plot them here.
### Feel free to use as many code cells as needed.

In [ ]:
import glob
import cv2
import matplotlib.image as mpimg

img_folder = 'onlineImgs/'
testImgs = []
colorImgs = []
testLabels = [11, 25, 13,15, 12,17]
fig = plt.figure(figsize=(16,4))

images = sorted(glob.glob('onlineImgs/*'))
j=0
# Step through the image list and resize the images to 32 x 32
for idx, fname in enumerate(images):
    img = mpimg.imread(fname)

    imgL = cv2.resize(img, (100, 100), interpolation = cv2.INTER_AREA)
    img = cv2.resize(img, (32, 32), interpolation = cv2.INTER_AREA)
    sub=plt.subplot(2,7,j+1)
    sub.set_title(str(signName.loc[testLabels[j]].ClassId)+": "+str(signName.loc[testLabels[j]].SignName), fontsize=8)
    sub.axis("off")
    j=j+1
    # plot the higher quality image
    plt.imshow(imgL)
    sub=plt.subplot(2,7,j+7)
    #as swell as the reduced size image of 32 x 32 pixels, as the test dataset
    sub.axis("off")
    plt.imshow(img)
    colorImgs.append(imgL)
    testImgs.append(img)


testImgs = preProcess(np.asarray(testImgs))

plt.savefig('outputImgs/3_testImgs.jpg')

### Predict the Sign Type for Each Image & Analyze the Performance

In [ ]:
### Run the predictions here and use the model to output the prediction for each image.
### Make sure to pre-process the images with the same pre-processing pipeline used earlier.
### Feel free to use as many code cells as needed.

In [ ]:
#use the trained model to predict the accuracy on 6 new images

with tf.Session() as sess:
    
    saver.restore(sess, "lenet")
    predLabels = sess.run (tf.argmax(logits,1), feed_dict={x: testImgs, y: testLabels, keepProb:1})
    print()
    print("         actual predicted")
    for i in range(len(testLabels)):
        if predLabels[i]==testLabels[i]:
            print("Correct!!  "+str(testLabels[i])+"      "+str(predLabels[i])+": "+str(signName.loc[predLabels[i]].SignName))
        else:
            print("Oops~     "+str(testLabels[i])+"      "+str(predLabels[i])+": "+str(signName.loc[predLabels[i]].SignName))
    print()            
    accuracy = evaluate(testImgs, predLabels)
    print("Online Images Predicted Accuracy = {:.3f}".format(accuracy))


### Output Top 5 Softmax Probabilities For Each Image Found on the Web

For each of the new images, print out the model's softmax probabilities to show the **certainty** of the model's predictions (limit the output to the top 5 probabilities for each image). [`tf.nn.top_k`](https://www.tensorflow.org/versions/r0.12/api_docs/python/nn.html#top_k) could prove helpful here. 

The example below demonstrates how tf.nn.top_k can be used to find the top k predictions for each image.

`tf.nn.top_k` will return the values and indices (class ids) of the top k predictions. So if k=3, for each sign, it'll return the 3 largest probabilities (out of a possible 43) and the correspoding class ids.

Take this numpy array as an example. The values in the array represent predictions. The array contains softmax probabilities for five candidate images with six possible classes. `tf.nn.top_k` is used to choose the three classes with the highest probability:

```
# (5, 6) array
a = np.array([[ 0.24879643,  0.07032244,  0.12641572,  0.34763842,  0.07893497,
         0.12789202],
       [ 0.28086119,  0.27569815,  0.08594638,  0.0178669 ,  0.18063401,
         0.15899337],
       [ 0.26076848,  0.23664738,  0.08020603,  0.07001922,  0.1134371 ,
         0.23892179],
       [ 0.11943333,  0.29198961,  0.02605103,  0.26234032,  0.1351348 ,
         0.16505091],
       [ 0.09561176,  0.34396535,  0.0643941 ,  0.16240774,  0.24206137,
         0.09155967]])
```

Running it through `sess.run(tf.nn.top_k(tf.constant(a), k=3))` produces:

```
TopKV2(values=array([[ 0.34763842,  0.24879643,  0.12789202],
       [ 0.28086119,  0.27569815,  0.18063401],
       [ 0.26076848,  0.23892179,  0.23664738],
       [ 0.29198961,  0.26234032,  0.16505091],
       [ 0.34396535,  0.24206137,  0.16240774]]), indices=array([[3, 0, 5],
       [0, 1, 4],
       [0, 5, 1],
       [1, 3, 5],
       [1, 4, 3]], dtype=int32))
```

Looking just at the first row we get `[ 0.34763842,  0.24879643,  0.12789202]`, you can confirm these are the 3 largest probabilities in `a`. You'll also notice `[3, 0, 5]` are the corresponding indices.

In [ ]:
### Print out the top five softmax probabilities for the predictions on the German traffic sign images found on the web. 
### Feel free to use as many code cells as needed.

In [ ]:
softmaxLogit = tf.nn.softmax(logits)
softmaxTopFiveIn = tf.nn.top_k(softmaxLogit, k=5)
with tf.Session() as sess:
    saver.restore(sess, "lenet")
    softmaxLogits = sess.run (softmaxLogit, feed_dict={x: testImgs, y: testLabels, keepProb:1})
    softmaxTopFive = sess.run (softmaxTopFiveIn, feed_dict={x: testImgs, y: testLabels, keepProb:1})
    print(softmaxTopFive)


In [ ]:
# it might be easier to see the softmax probabilities graphically
fig, axs = plt.subplots(3,4, figsize=(16, 9))
axs = axs.ravel()
    
for i in range(len(testImgs)*2):
    if i%2 == 0:
        axs[i].axis('off')
        axs[i].imshow(colorImgs[i//2])
    else:
        axs[i].bar(np.arange(n_classes), softmaxLogits[(i-1)//2]) 
        axs[i].set_ylabel('Softmax probability')    
        
plt.savefig('outputImgs/3_softmax.jpg')

### Project Writeup

Once you have completed the code implementation, document your results in a project writeup using this [template](https://github.com/udacity/CarND-Traffic-Sign-Classifier-Project/blob/master/writeup_template.md) as a guide. The writeup can be in a markdown or pdf file. 

> **Note**: Once you have completed all of the code implementations and successfully answered each question above, you may finalize your work by exporting the iPython Notebook as an HTML document. You can do this by using the menu above and navigating to  \n",
    "**File -> Download as -> HTML (.html)**. Include the finished document along with this notebook as your submission.

---

## Step 4 (Optional): Visualize the Neural Network's State with Test Images

 This Section is not required to complete but acts as an additional excersise for understaning the output of a neural network's weights. While neural networks can be a great learning device they are often referred to as a black box. We can understand what the weights of a neural network look like better by plotting their feature maps. After successfully training your neural network you can see what it's feature maps look like by plotting the output of the network's weight layers in response to a test stimuli image. From these plotted feature maps, it's possible to see what characteristics of an image the network finds interesting. For a sign, maybe the inner network feature maps react with high activation to the sign's boundary outline or to the contrast in the sign's painted symbol.

 Provided for you below is the function code that allows you to get the visualization output of any tensorflow weight layer you want. The inputs to the function should be a stimuli image, one used during training or a new one you provided, and then the tensorflow variable name that represents the layer's state during the training process, for instance if you wanted to see what the [LeNet lab's](https://classroom.udacity.com/nanodegrees/nd013/parts/fbf77062-5703-404e-b60c-95b78b2f3f9e/modules/6df7ae49-c61c-4bb2-a23e-6527e69209ec/lessons/601ae704-1035-4287-8b11-e2c2716217ad/concepts/d4aca031-508f-4e0b-b493-e7b706120f81) feature maps looked like for it's second convolutional layer you could enter conv2 as the tf_activation variable.

For an example of what feature map outputs look like, check out NVIDIA's results in their paper [End-to-End Deep Learning for Self-Driving Cars](https://devblogs.nvidia.com/parallelforall/deep-learning-self-driving-cars/) in the section Visualization of internal CNN State. NVIDIA was able to show that their network's inner weights had high activations to road boundary lines by comparing feature maps from an image with a clear path to one without. Try experimenting with a similar test to show that your trained network's weights are looking for interesting features, whether it's looking at differences in feature maps from images with or without a sign, or even what feature maps look like in a trained network vs a completely untrained one on the same sign image.

<figure>
 <img src="visualize_cnn.png" width="380" alt="Combined Image" />
 <figcaption>
 <p></p> 
 <p style="text-align: center;"> Your output should look something like this (above)</p> 
 </figcaption>
</figure>
 <p></p> 


In [ ]:
### Visualize your network's feature maps here.
### Feel free to use as many code cells as needed.

# image_input: the test image being fed into the network to produce the feature maps
# tf_activation: should be a tf variable name used during your training procedure that represents the calculated state of a specific weight layer
# activation_min/max: can be used to view the activation contrast in more detail, by default matplot sets min and max to the actual min and max values of the output
# plt_num: used to plot out multiple different weight feature map sets on the same block, just extend the plt number for each new feature map entry

def outputFeatureMap(image_input, tf_activation, activation_min=-1, activation_max=-1 ,plt_num=1):
    # Here make sure to preprocess your image_input in a way your network expects
    # with size, normalization, ect if needed
    # image_input =
    # Note: x should be the same name as your network's tensorflow data placeholder variable
    # If you get an error tf_activation is not defined it may be having trouble accessing the variable from inside a function
    activation = tf_activation.eval(session=sess,feed_dict={x : image_input})
    featuremaps = activation.shape[3]
    plt.figure(plt_num, figsize=(15,15))
    for featuremap in range(featuremaps):
        plt.subplot(6,8, featuremap+1) # sets the number of feature maps to show on each row and column
        plt.title('FeatureMap ' + str(featuremap)) # displays the feature map number
        if activation_min != -1 & activation_max != -1:
            plt.imshow(activation[0,:,:, featuremap], interpolation="nearest", vmin =activation_min, vmax=activation_max, cmap="gray")
        elif activation_max != -1:
            plt.imshow(activation[0,:,:, featuremap], interpolation="nearest", vmax=activation_max, cmap="gray")
        elif activation_min !=-1:
            plt.imshow(activation[0,:,:, featuremap], interpolation="nearest", vmin=activation_min, cmap="gray")
        else:
            plt.imshow(activation[0,:,:, featuremap], interpolation="nearest", cmap="gray")